# Optimising simulations for a generic loss function

This notebook showcases optimising a design for some generic loss function $\mathcal{L}$ using an event loop calling **Elmer** or **HFSS**.
As such, this notebook should be run on a machine with the simulation software, KLayout and necessary Python packages installed.
This is similar to Ansys Optimetrics but more general due to supporting arbitrary KQCircuits geometry,
Elmer and, any search algorithms and schedulers.
The current implementation requires installing [`ray`](https://docs.ray.io/en/releases-2.0.0/index.html)
```bash
pip install "ray[tune]"==2.1.0
```
and the optimisers generally demand additional manual installation. For example, this notebook employs [`Optuna`](https://optuna.readthedocs.io/en/stable/) and we need to also install:
```bash
pip install optuna==3.0.4 botorch==0.7.3 scikit-learn==1.1.3 plotly==5.11.0
```
Newer versions may work but these are the ones the notebook was written with.

## Workflow

1. Supply your ``Simulation`` class, `sim_parameters`, and `export_parameters` as usual in the [Simulation settings](#simulation-settings)
2. Implement fetching simulation results for your use case and a meaningful loss function $\mathcal{L}$ in the [Loss settings](#loss-settings) section.
3. Specify the variables that can be varied by the optimiser and their ranges in [Optimisation settings](#optimisation-settings). See [Ray Tune - Random Distributions API](https://docs.ray.io/en/releases-2.0.0/tune/api_docs/search_space.html?highlight=quniform#random-distributions-api) for possible (and custom) distributions.
4. Select your optimiser and its settings. You can also try using a simpler framework, such as, `scipy`, `pytorch`, `tensorflow` etc. See [Ray Tune - Search Algorithms](https://docs.ray.io/en/releases-2.0.0/tune/api_docs/suggestion.html) for algorithms with support out-of-the-box.
5. Run the optimiser (`ray` tuning)
6. View the results! You can manipulate the DataFrame, or use [Tensorboard](https://www.tensorflow.org/tensorboard) to view the data in the same folder as this notebook:
   ```bash
   pip install tensorboard
   tensorboard --logdir=./ray_results
   ```


The _toy example_ implemented in this notebook is optimising the qubit $C_\Sigma$ ``DoublePads`` to be $65\,\text{fF}$.

In [1]:
import os
import re
import json
import shutil
import subprocess
import importlib
from copy import deepcopy
from pathlib import Path
from functools import partial
from datetime import datetime

import pandas as pd
import ray
import ray.air
import ray.air.session
from ray import tune

from kqcircuits.pya_resolver import pya
from kqcircuits.defaults import STARTUPINFO
from kqcircuits.util.geometry_json_encoder import GeometryJsonEncoder
from kqcircuits.simulations.export.ansys.ansys_export import export_ansys
from kqcircuits.simulations.export.elmer.elmer_export import export_elmer
from kqcircuits.simulations.export.simulation_export import export_simulation_oas
from kqcircuits.util.export_helper import create_or_empty_tmp_directory, get_active_or_new_layout

if os.environ.get('KQC_TMP_PATH') is None:
    os.environ['KQC_TMP_PATH'] = os.getcwd()
original_kqc_tmp_path = os.environ['KQC_TMP_PATH']

### Please set the location of your interpreter for `.sh` files

Some examples are given below

In [2]:
# Please set the location of your interpreter for `.sh` files, some examples are given below.
# The interpreter will be called with `-c SCRIPT.sh`.
# If this does not work, try adjusting the `subprocess.run` call later in the notebook
sh_interpreter = 'sh'
sh_interpreter = shutil.which('bash') if os.name != 'nt' else 'wsl -e bash'  # use WSL on Windows
sh_interpreter = r'C:\Program Files\Git\bin\bash.exe'

if shutil.which(sh_interpreter) is None:
    raise FileNotFoundError(f"Your shell `{sh_interpreter=}` is not found! Please set it to a working shell.")

tool = ['ansys', 'elmer'][1]
# This needs to be only as high as the number of licenses when using Ansys
n_workers = 4 if tool == 'elmer' else 1  # first-level parellelisation
n_processes = 4 if tool == 'elmer' else 5  # second-level parellelisation

## Implement helper functions

In [3]:
def create_and_export_sim(params, sim_class, sim_parameters, export_parameters, i):
    """Creates and exports a :class:`.Simulation` with given parameters.

    Arguments:
        params: Iteration-specific parameters to use. Combined with ``sim_parameters``.
        sim_class (Simulation): Given simulation. Should be loadable from a module (not anonymous).
        sim_parameters (dict): Dict of nominal simulation parameters.
        export_parameters (dict): Dict of simulation export parameters. May be for Ansys or Elmer
        i (str): Unique identifier for this iteration.

    Returns:
        dir_path: Path to exported simulations folder.
    """

    # Find corresponding class, has to be done this way to avoid pickling
    module = importlib.import_module(sim_class[0])
    sim_class = getattr(module, sim_class[1])

    sim_parameters['name'] = sim_parameters['name'] + f"_opt_{i}"
    dir_path = create_or_empty_tmp_directory(sim_parameters['name'])
    export_parameters = {'path': dir_path, **export_parameters}

    # Get layout
    layout = get_active_or_new_layout()

    # Nominal simulation
    simulations = [sim_class(layout, **{
        **sim_parameters,
        **params
    })]

    if 'ansys_tool' in export_parameters:
        export_ansys(simulations, **export_parameters)
    else:
        export_elmer(simulations, **export_parameters)

    # Write oas files
    export_simulation_oas(simulations, dir_path)

    return dir_path

Implement small serializer for DBoxes, as it is often needed in `sim_parameters`.

In [4]:
def dbox_serializer(dbox):
    return GeometryJsonEncoder.encode_geometry(dbox)

def dbox_deserializer(o):
    return pya.DBox(
        pya.DPoint(o[0][0], o[0][1]), pya.DPoint(o[1][1], o[1][1])
    )

ray.util.register_serializer(pya.DBox, serializer=dbox_serializer, deserializer=dbox_deserializer)

## Simulation settings

Specify simulation export settings as normal

In [5]:
# pylint: disable=invalid-name,wrong-import-position
from kqcircuits.simulations.double_pads_sim import DoublePadsSim

sim_class = DoublePadsSim

# Nominal simulation parameters
sim_parameters = {
    'name': re.sub(r'(?<!^)(?=[A-Z])', '_', sim_class.__name__).lower(),  # convert to snake_case
    'use_internal_ports': True,
    'use_ports': True,
    'internal_island_ports': True,
    'box': pya.DBox(pya.DPoint(0,0), pya.DPoint(2000, 2000)),
    'face_stack': ['1t1'],
}

#########
# Ansys #
#########

export_parameters_ansys_q3d = {
    'ansys_tool': 'q3d',
    'exit_after_run': True,
    'percent_error': 0.3,
    'minimum_converged_passes': 2,
    'maximum_passes': 20,
}

# These are not used for capacitance computations
export_parameters_ansys_epr = {
    'ansys_tool': 'eigenmode',
    'exit_after_run': True,
    'max_delta_f': 0.05,

    # do two passes with tight mesh
    'gap_max_element_length': 24,
    'maximum_passes': 15,
    'minimum_passes': 1,
    'minimum_converged_passes': 2,

    # lossy eigenmode simulation settings
    'n_modes': 1,
    'frequency': 0.5,  # minimum allowed eigenfrequency
    'simulation_flags': ['pyepr'],

    # run T1 analysis with pyEPR between simulations
    'intermediate_processing_command': 'python "scripts/t1_estimate.py"',
    'participation_sheet_distance': 5e-3,  # in µm

    'substrate_loss_tangent': 1e-6,
    'dielectric_surfaces': {
        'layerMA': {
            'tan_delta_surf': 0.001,  # loss tangent
            'th': 4.8e-9,  # thickness in m
            'eps_r': 10,  # relative permittivity
        },
        'layerMS': {
            'tan_delta_surf': 0.001,
            'th': 0.3e-9,  # estimate worst case
            'eps_r': 10,
        },
        'layerSA': {
            'tan_delta_surf': 0.001,
            'th': 2.4e-9,
            'eps_r': 10,
        }
    },
}

#########
# Elmer #
#########

export_parameters_elmer = {
    'tool': 'capacitance',
    'workflow': {
        'python_executable': 'python',  # use 'kqclib' when using singularity
        'run_gmsh_gui': False,
        'elmer_n_processes': n_processes,  # -1 means all the physical cores
        'gmsh_n_threads': n_processes,
    },
    'mesh_size': {
        'global_max': 60.,
        'gap&signal': [2., 4.],
        'gap&ground': [2., 4.],
        'port': [1., 4.],
    },
    'linear_system_method': 'mg',
    'p_element_order': 2,
}

We choose to use Elmer for the capacitance simulations

In [6]:
export_parameters = export_parameters_elmer  # use Elmer
# export_parameters = export_parameters_ansys_epr  # use Ansys + pyEPR

## Loss settings

These need to be adjusted to fit your specific use case.
The optimiser is set to minimise. If you want to maximise, you can modify $\mathcal{L} \to -\mathcal{L}$.
Furthermore, the simplest way to attain a specific value is to set the loss function to
$$
\mathcal{L}(x) = \left| x_\text{desired} - x \right|
$$
This is not differentiable at $\mathcal{L}(x_\text{desired})$, but appears to work decently.
A common alternative is to use the mean squared error ($L_2$-norm).

In [7]:
def load_results_Q(dir_path):
    """Function to load simulation results. Input is simulation folder path."""
    result_file = list(Path(dir_path).rglob('Qdata*.csv'))[0]
    df = pd.read_csv(result_file)
    return df.iloc[0].to_dict()


def loss_Q(data):
    r"""Loss function :math:`$\mathcal(L)` to minimum loss :math:`\delta`."""
    Q = data['Q_total']
    return 1 / Q, Q


def load_results_capacitance(dir_path):
    """Function to load simulation results. Input is simulation folder path."""
    result_file = list(Path(dir_path).rglob('*_project_results.json'))[0]
    with open(result_file) as fp:
        data = json.load(fp)
    return data


def loss_C_Sigma_two_islands(data, C):
    r""" Data argument is a dict with `CMatrix` key of a 3x3 capacitance matrix with a coupler as the last port.
    See “Design and modelling of long-coherence qubits using energy participation ratios”, Niko Savola, Appendix B.

    Returns:
        loss: MSE loss to target ``C``
        C_Sigma: absolute value for :math:`C_\Sigma`
    """
    CMatrix = data['CMatrix']

    C_theta = (CMatrix[0][0] + CMatrix[0][2] + CMatrix[1][1] + CMatrix[1][2])
    C_q = CMatrix[0][1] + ((CMatrix[0][0] + CMatrix[0][2]) * (CMatrix[1][1] + CMatrix[1][2])) / C_theta
    C_r = CMatrix[0][2] + CMatrix[1][2] + CMatrix[2][2] - (CMatrix[0][2] + CMatrix[1][2]) ** 2 / C_theta
    C_qr = (CMatrix[0][2] * CMatrix[1][1] - CMatrix[1][2] * CMatrix[0][0]) / C_theta

    C_sigma = C_q - C_qr ** 2 / C_r
    return ((C - C_sigma) * 1e15) ** 2, C_sigma


def loss_capacitance(data, C, i=0, j=0):
    r"""Loss function :math:`$\mathcal(L)` to get certain capacitance.

    Arguments:
        data: simulation results dictionary containing ``CMatrix``
        C: desired capacitance in farads
        i: first index for the desired capacitance from the ``CMatrix``
        j: second index for the desired capacitance from the ``CMatrix``

    Returns:
        loss: The computed loss
        x: Value with which the loss was computed

    Usage:

        .. code-block:: python

            from functools import partial
            loss = partial(lossC, C=50e-15, i=1, j=0)
            # Do the following inside the event loop
            results = load_results(dir_path)
            score = loss(results)
    """
    C_sim = data['CMatrix'][i][j]
    return ((C - C_sim) * 1e15) ** 2, C_sim  # Optimiser might get confused by small numbers

Select your loss function to use

In [8]:
load_results = load_results_capacitance

# loss = partial(loss_capacitance, C=100e-15, i=0, j=1)
loss = partial(loss_C_Sigma_two_islands, C=65e-15)

# load_results = load_results_Q
# loss = loss_Q

## Optimisation settings

Specify parameter search spaces, optimiser (such as ``HyperOptSearch``), possible scheduler, maximum iterations,
and parallel workers (only with Elmer).

In [9]:
search_config = {
    'params': {
        'island1_extent_[0]': tune.uniform(200, 700),
        'island1_extent_[1]': tune.uniform(50, 500),

        'island2_extent_[0]': tune.uniform(200, 700),
        'island2_extent_[1]': tune.uniform(50, 300),

        'coupler_extent[0]': tune.uniform(40, 500),
        'coupler_extent[1]': tune.uniform(20, 40),
        'coupler_offset': tune.uniform(5, 50),

        'ground_gap_[0]': tune.uniform(550, 1000),
        'ground_gap_[1]': tune.uniform(400, 1000),

        'squid_offset': tune.uniform(-20, 0),
        'island1_r': tune.uniform(0, 50),
        'island2_r': tune.uniform(0, 50),
    },
}

def constraints(trial):
    """ Return Sequence of (soft) constraints for the optimiser. Positive values are punished and <= 0 are ok.
    See optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.BoTorchSampler.html for details.
    """
    c0 = trial.params['params/ground_gap'] + trial.params['params/r_outer'] - 150
    c1 = trial.params['params/r_inner'] - trial.params['params/r_outer']  # negative is ok
    return [c0, c1]  # tries to optimise equal to 0

# Avoid more pickling of KLayout classes and don't show these as search params
create_and_export = partial(
    create_and_export_sim,
    sim_parameters=sim_parameters,
    export_parameters=export_parameters,
    sim_class=(sim_class.__module__, sim_class.__name__),  # we avoid pickling the class by loading this way
)

# Import chosen optimiser and scheduler. NB: Selecting a new optimiser often requires you to install a separate package
# See https://docs.ray.io/en/releases-2.0.0/tune/api_docs/suggestion.html

# pylint: disable=wrong-import-position,ungrouped-imports
from ray.tune.search.optuna import OptunaSearch
import optuna
import joblib  # shold come with Optuna


tune_config = tune.TuneConfig(
    metric=(metric := 'score'),
    mode=(mode := 'min'),
    search_alg=(optuna_search := OptunaSearch(
        # If you want soft constraints
        sampler=optuna.integration.BoTorchSampler(constraints_func=constraints),
        # sampler=optuna.integration.BoTorchSampler(),
        metric=[metric],  #, 'outer_island_width'],
        mode=[mode]  #, 'min'],
    )),
    ## Examples of other possible optimisers (search algorithms)
    # search_alg=BayesOptSearch(),
    # search_alg=HEBOSearch(),
    # search_alg=DragonflySearch(
    #     metric='score',
    #     mode='min',
    #     optimizer='bandit',
    #     domain='Euclidean'
    # ),
    max_concurrent_trials=n_workers,  # Has to be 1 for Ansys, may be more for Elmer
    num_samples=50,  # max iterations, can be -1 for infinite
    time_budget_s=0,  # Time after which optimisation is stopped. May be useful along with ``num_samples=-1``.
)

C:\Users\NikoSavola\AppData\Local\Temp\ipykernel_18148\3479706229.py:52: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler=optuna.integration.BoTorchSampler(constraints_func=constraints),


In [10]:
def event_loop(config):
    config_edited = deepcopy(config)

    # Combine parameters with the following format to single array param:
    #   param_[0] + param_[1] -> param = [param_[0], param_[1]]
    arr_match = r'[a-z_]+_\[(\d)\]$'
    for key in list(config['params'].keys()):
        if re.match(arr_match, key):
            arr_key = re.sub(r'_\[(\d)\]$', '', key)
            config_edited['params'][arr_key] = config_edited['params'].get(arr_key, []) + [config_edited['params'][key]]

    dir_path = create_and_export(i=ray.air.session.get_trial_id(), **config_edited)
    use_ansys = 'ansys_tool' in create_and_export.keywords['export_parameters']

    subprocess.run(
        ['simulation.bat'] if use_ansys else [sh_interpreter, '-c', './simulation.sh'],
        cwd=dir_path,
        shell=True,
        check=True,
        startupinfo=STARTUPINFO
    )

    results = load_results(dir_path)
    score, x = loss(results)
    return {'score': score, 'value': x}
    # For multi-objective, just return the different loss functions and specify in `metric`, e.g.
    # from math import prod
    # return {'score': score, 'value': x, 'island1_size': math.prod(config['params']['island1_extent'])}

## Run optimisation

Start a Tune run and print the results.

In [11]:
# Set folder for optimisation files
os.environ['KQC_TMP_PATH'] = (
    Path(original_kqc_tmp_path) / (sim_class.__name__ + '_' + str(datetime.now()))\
        .replace(' ', '_').replace(':', '.')
).as_posix()

tuner = tune.Tuner(
    tune.with_resources(event_loop, {"cpu": n_processes}),  # maximum resources given to worker
    param_space=search_config,
    tune_config=tune_config,
    run_config=ray.air.RunConfig(
        local_dir=os.environ['KQC_TMP_PATH'] + "/ray_results",  # needed for tensorboard
        checkpoint_config=ray.air.CheckpointConfig(
            checkpoint_frequency=1
        ),
        log_to_file=True
    )
)

# For resuming a run
# tuner = tune.Tuner.restore(os.environ['KQC_TMP_PATH'] + "/ray_results" + "/some_folder", restart_errored=False)

results = tuner.fit()

# This is specific only for the Optuna optimiser, we also save the 'study' object
study_file = Path(results._experiment_analysis.runner_data()['_local_checkpoint_dir']) / "study.pkl"
if study_file.exists():
    print(f'Loading study from {study_file}')
    study = joblib.load(study_file)
else:
    print(f'Saving study to {study_file}')
    study = optuna_search._ot_study
    joblib.dump(study, study_file)

os.environ['KQC_TMP_PATH'] = original_kqc_tmp_path

Saving study to D:\LocalSimulations\DoublePadsSim_2023-02-23_09.29.03.469245\ray_results\event_loop_2023-02-23_09-29-03\study.pkl


This is specific only for the Optuna optimiser

In [12]:
# Default 'importance' metric is fANOVA, see optuna.importance.FanovaImportanceEvaluator
fig = optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0])
fig.show()

fig = optuna.visualization.plot_parallel_coordinate(study, target=lambda t: t.values[0])
fig.show()

try:
    fig = optuna.visualization.plot_pareto_front(study)  # or optuna.visualization.matplotlib.plot_pareto_front(study)
    fig.show()
except ValueError as e:
    print('Not multi-objective.', e)

C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_utils.py:70: UserWarning: `target` is specified, but `target_name` is the default value, 'Objective Value'.
  warnings.warn(


C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_utils.py:70: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



Not multi-objective. `plot_pareto_front` function only supports 2 or 3 objective studies when using `targets` is `None`. Please use `targets` if your objective studies have more than 3 objectives.


In [13]:
df = results.get_dataframe()
best = results.get_best_result(metric="score", mode="min")

display(df.filter(regex='(score|value|time_this|config)'))
print(f"{best.metrics['value']=}", best.config)

,score,value,time_this_iter_s,config/params/coupler_extent[0],config/params/coupler_extent[1],config/params/coupler_offset,config/params/ground_gap_[0],config/params/ground_gap_[1],config/params/island1_extent_[0],config/params/island1_extent_[1],config/params/island1_r,config/params/island2_extent_[0],config/params/island2_extent_[1],config/params/island2_r,config/params/squid_offset
0,10.482479,6.176233e-14,93.964783,209.508885,20.257585,27.908284,980.836474,500.147391,612.197751,299.446062,39.717320,347.268027,158.301448,39.224498,-1.858705e+01
1,8.791125,6.203502e-14,91.787727,248.199973,28.410353,30.168589,848.421707,567.943640,230.710236,396.079244,1.188055,565.924384,107.273236,42.363652,-1.005205e+01
2,3.077639,6.324568e-14,91.767391,436.940031,28.407880,27.418650,779.464603,447.486883,684.073920,96.301390,34.595053,267.753046,58.573526,21.208529,-1.075408e+01
3,11.471736,6.161300e-14,91.759390,206.252459,35.671084,44.978300,978.270403,625.973263,422.699320,170.804017,40.336896,363.347414,115.216341,7.119852,-1.422883e+01
4,4.009674,6.299758e-14,94.803180,288.069242,23.450317,19.086692,667.891683,733.465205,643.740915,91.311781,7.973144,297.323156,218.705652,17.416241,-5.318944e+00
5,9.357473,6.194100e-14,95.555332,454.863522,33.498393,29.036728,939.156859,960.560671,666.906152,211.413420,26.550374,489.228789,133.783461,5.640993,-1.793807e+01
6,10.288915,6.179237e-14,95.441624,312.041548,39.525873,36.733802,993.274508,812.575188,604.205713,204.201544,21.351640,655.366147,192.270753,20.050858,-2.268976e+00
7,13.623701,6.130897e-14,93.026208,191.699309,35.519139,38.462601,990.175799,567.110876,294.865395,358.749657,34.722616,618.337660,184.380039,38.642559,-1.908002e+01
8,11.574816,6.159782e-14,90.947189,128.780401,31.385516,29.271786,835.945710,635.084194,495.912807,54.309898,47.842677,467.245824,184.121508,0.887890,-1.989711e+01
9,1.608149,6.373187e-14,90.539839,237.675322,22.431663,13.329813,609.954560,522.537032,406.118551,241.213358,33.194390,571.839020,68.159437,10.140015,-1.074018e+00


best.metrics['value']=6.499659380176716e-14 {'params': {'island1_extent_[0]': 524.2909325188749, 'island1_extent_[1]': 189.39516609617917, 'island2_extent_[0]': 543.1565443954432, 'island2_extent_[1]': 64.75618358331897, 'coupler_extent[0]': 237.36599856925082, 'coupler_extent[1]': 23.226950733465802, 'coupler_offset': 11.285557368465366, 'ground_gap_[0]': 552.0120393443678, 'ground_gap_[1]': 486.00491041980274, 'squid_offset': -0.9753492573896594, 'island1_r': 38.83005701863951, 'island2_r': 11.0749634538422}}
